# Spatial Analysis of Vaccination Rate

## Introduction
COVID-19 vaccination rates analysis. This notebook provides the base layer of COVID-19 vaccination rates in the United States. More layers could be added to the base layer to examine the relationship between the vaccination rate and a new feature. 

## Data
1.  Vaccination 
* Percent of adults fully vaccinated (state level): https://data.cdc.gov/resource/unsk-b7fc.json
* Percent of adults fully vaccinated (county level): https://data.cdc.gov/resource/8xkx-amqh.json
2.  Spatial
* [U.S. State](https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip)
* [County](https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip)
* [ZIP Code Tabulation Areas - 5-Digits](https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_zcta510_500k.zip)
* [ZIP Code Tabulation Areas - 3-Digits](https://www2.census.gov/geo/tiger/PREVGENZ/zt/z300shp/z399_d00_shp.zip)

Use the appropriate API address for retriving vaccination data. Click the links to download the spatial files. 

*To use different spatial layers, navigate to [Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html) to download spatial data.*

## Which level of spatial data to use? 
| Spatial Data/Layers | Zip5 | Zip3 | County | State |
|---------------------|------|------|--------|-------|
| Geo file            | 1    | 1    | 1      | 1     |
| Vaccination         | 0    | 0    | 1      | 1     |
| Market Share        | 0    | 1    | 0      | 1     |


The granularity of the vaccination rate and the additional feature should be the same to ensure the data is comparable. Choose to downsample or upsample the data to achieve the equal spatial granularity if possible. 

You could create crosswalk from zipcode to other spatial granularity using the file below. 
*  [HUD USPS ZIP Code Crosswalk files](https://www.huduser.gov/portal/datasets/usps_crosswalk.html#codebook)
The crosswalk files contains ratio of the residential and business addresses that fall in different counties for each zipcode. We could possibly use the business addresses to estimate the distribution of market share in the counties. 

## Import dependencies

In [1]:
import pandas as pd
from sodapy import Socrata
from urllib.request import urlopen 
import json 
import geopandas
import folium
from geopy.geocoders import Nominatim

## Import vaccination data

In [2]:
# get updated vaccination data

# CDC - use Socrata API to get updated county level vaccination data.

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# use saved credential files from 'credential.txt'
with open('credential.txt') as f:
    lines = f.readlines()

cred = json.loads(lines[0])
client = Socrata("data.cdc.gov",
                  cred['token'],
                  username=cred['username'],
                  password=cred['password'])

In [3]:
# First 2000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
# county: 8xkx-amqh
# state: unsk-b7fc

# state vaccination rate
dates = "'2021-01-31T00:00:00.000','2021-02-28T00:00:00.000','2021-03-31T00:00:00.000',\
        '2021-04-30T00:00:00.000','2021-05-31T00:00:00.000','2021-06-30T00:00:00.000',\
        '2021-07-31T00:00:00.000','2021-08-31T00:00:00.000','2021-09-30T00:00:00.000',\
        '2021-10-31T00:00:00.000'"
# dates = "'2021-01-31T00:00:00.000','2021-02-28T00:00:00.000','2021-03-31T00:00:00.000','2021-04-30T00:00:00.000','2021-05-31T00:00:00.000','2021-06-30T00:00:00.000'"
results = client.get("unsk-b7fc", 
                     query="SELECT date, location, series_complete_pop_pct WHERE date IN ({0})".format(dates))
state_results_df = pd.DataFrame.from_records(results) # Convert to pandas DataFrame
display(state_results_df.head())

,date,location,series_complete_pop_pct
0,2021-01-31T00:00:00.000,OH,0
1,2021-01-31T00:00:00.000,NV,0
2,2021-01-31T00:00:00.000,MA,0
3,2021-01-31T00:00:00.000,KY,0
4,2021-01-31T00:00:00.000,OK,0


In [4]:
##Get county level vaccination rate
results = client.get("8xkx-amqh", limit=3300)
county_results_df = pd.DataFrame.from_records(results) # Convert to pandas DataFrame
display(county_results_df.head())

,date,fips,mmwr_week,recip_county,recip_state,series_complete_pop_pct,series_complete_yes,series_complete_12plus,series_complete_12pluspop,series_complete_18plus,...,svi_ctgy,series_complete_pop_pct_svi,series_complete_12pluspop_pct_svi,series_complete_18pluspop_pct_svi,series_complete_65pluspop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,series_complete_12pluspop_pct_ur_equity,series_complete_18pluspop_pct_ur_equity,series_complete_65pluspop_pct_ur_equity
0,2021-11-14T00:00:00.000,13265,46,Taliaferro County,GA,22.1,340,340,25.1,317,...,D,13,13,13,13,Non-metro,5,5,5,5
1,2021-11-14T00:00:00.000,21219,46,Todd County,KY,38.3,4705,4705,46.2,4504,...,D,14,15,15,16,Non-metro,6,7,7,8
2,2021-11-14T00:00:00.000,26047,46,Emmet County,MI,68.5,22875,22873,77.6,21581,...,A,4,4,4,4,Non-metro,8,8,8,8
3,2021-11-14T00:00:00.000,40079,46,Le Flore County,OK,35.5,17684,17684,41.9,16861,...,D,14,15,15,15,Metro,2,3,3,3
4,2021-11-14T00:00:00.000,16057,46,Latah County,ID,50.1,20098,NaN,NaN,19898,...,A,4,NaN,4,4,Non-metro,8,NaN,8,8


## Import spatial data file

In [5]:
# read in shp file as GeoPandas dataframe, 
# process GeoPandas dataframe
# convert GeoPandas to json files
# use the json file with folium.

# read state shape file
zipfile = "zip://cb_2018_us_state_500k.zip/cb_2018_us_state_500k.shp"
state_geo = geopandas.read_file(zipfile)
state_geo.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."


In [6]:
# read county shape file
zipfile = "zip://cb_2018_us_county_500k.zip/cb_2018_us_county_500k.shp"
county_geo = geopandas.read_file(zipfile)
county_geo.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."


In [7]:
# read zip5 shape file
# zipfile = "zip://cb_2018_us_zcta510_500k.zip/cb_2018_us_zcta510_500k.shp"
# read zip3 shape file
# zipfile = "zip://z399_d00_shp.zip/z399_d00.shp"
# zip3 = geopandas.read_file(zipfile)

## Data cleaning

In [8]:
# state data
state_vac = state_results_df.astype({'date':'datetime64[ns]','series_complete_pop_pct':'float64'})

# fill NAs in state_vaccination data with the most recent rate by location
state_vac["series_complete_pop_pct"] = state_vac.groupby('location')['series_complete_pop_pct'].apply(lambda x:x.fillna(x.max()))
state_vac.dropna(subset=["series_complete_pop_pct"], inplace=True) # drop rows which still have NA state_vaccination rate
state_vac = state_vac[state_vac.date == state_vac.date.max()] # take the most up-to-date data
state_vac.replace('New York State','New York', inplace=True) # sync the state names on state_vac and state
display(state_vac.head())

# merge vaccination data with spatial data
state = state_geo.loc[:,['STUSPS','NAME','geometry']].merge(
                                                    state_vac[["location","series_complete_pop_pct"]],
                                                    left_on=['STUSPS'],
                                                    right_on=['location'],
                                                    how='left'
                                                    )
display(state.head())

,date,location,series_complete_pop_pct
585,2021-10-31,NV,53.0
586,2021-10-31,WA,63.5
587,2021-10-31,MH,34.7
588,2021-10-31,OH,51.9
589,2021-10-31,VT,71.2


,STUSPS,NAME,geometry,location,series_complete_pop_pct
0,MS,Mississippi,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ...",MS,45.7
1,NC,North Carolina,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ...",NC,52.7
2,OK,Oklahoma,"POLYGON ((-103.00257 36.52659, -103.00219 36.6...",OK,50.1
3,VA,Virginia,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ...",VA,63.1
4,WV,West Virginia,"POLYGON ((-82.64320 38.16909, -82.64300 38.169...",WV,41.0


In [9]:
# county data
county_vac = county_results_df[['date','mmwr_week','recip_county','recip_state','series_complete_pop_pct','fips']]
county_vac = county_vac.astype({'date':'datetime64[ns]',
                                'mmwr_week':'int64',
                                'series_complete_pop_pct':'float64'})
county_vac = county_vac[county_vac.date == county_vac.date.max()] # take data of the latest date.
display(county_vac.head())

# merge vaccination data with spatial data
county = county_geo.loc[:,['GEOID','NAME','geometry']].merge(
                                                    county_vac[["fips","recip_state","recip_county","series_complete_pop_pct"]],
                                                    left_on=['GEOID'],
                                                    right_on=['fips'],
                                                    how='left'
                                                    )
display(county.head())

,date,mmwr_week,recip_county,recip_state,series_complete_pop_pct,fips
0,2021-11-14,46,Taliaferro County,GA,22.1,13265
1,2021-11-14,46,Todd County,KY,38.3,21219
2,2021-11-14,46,Emmet County,MI,68.5,26047
3,2021-11-14,46,Le Flore County,OK,35.5,40079
4,2021-11-14,46,Latah County,ID,50.1,16057


,GEOID,NAME,geometry,fips,recip_state,recip_county,series_complete_pop_pct
0,21007,Ballard,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007,KY,Ballard County,34.3
1,21017,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017,KY,Bourbon County,51.2
2,21031,Butler,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031,KY,Butler County,41.1
3,21065,Estill,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065,KY,Estill County,46.4
4,21069,Fleming,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069,KY,Fleming County,46.6


In [10]:
# convert dataframes to json
state_geo = state.to_crs(epsg='4269').to_json()
county_geo = county.to_crs(epsg='4269').to_json()

## Create map

In [11]:
# State vaccination map

# basic map centered at United States
m = folium.Map(location = [37.0902,-95.7129], 
                   tiles='cartodbpositron',
                    zoom_start=3)

# quantile based coloring
bins = list(state["series_complete_pop_pct"].quantile([0, 0.05, 0.25, 0.5, 0.75, 0.95, 1]))

# state layer
state_layer = folium.Choropleth(
    geo_data=state_geo, # geo_data should be in json format
    name="State Level",
    data=state, # data should be in pandas.DataFrame format
    columns=["location", "series_complete_pop_pct"], # columns from data
    key_on="feature.properties.STUSPS", # key from geo_data that matches the first column given in the columns parameter
    fill_color="Blues", # colorbrewer
    fill_opacity=0.6,
    line_opacity=1,
    
    # use the following lines if coloring based on quantile/bins
    bins=bins,
    legend_name="Vaccination Rate (%)"
).add_to(m)

# tooltips
state_layer.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['location','series_complete_pop_pct'], 
                                   aliases=['State','Fully Vaccinated Ppl%'],
                                   style=('background-color: grey; color: white;')
                                   )
).add_to(m)

## save the map
# m.save("StateVaccinationMap.html")

In [12]:
# County vaccination map

# basic map centered at United States
m = folium.Map(location = [37.0902,-95.7129], 
                   tiles='cartodbpositron',
                    zoom_start=3)

# add state outline 
state_layer = folium.Choropleth(
    geo_data=state_geo, # geo_data should be in json format
    name="State Level",
    fill_opacity=0,
    line_opacity=1
).add_to(m)

# quantile based coloring
bins = list(county["series_complete_pop_pct"].quantile([0, 0.05, 0.25, 0.5, 0.75, 0.95, 1]))

# county layer
county_layer = folium.Choropleth(
    geo_data=county_geo, # geo_data should be in json format
    name="County Level",
    data=county, # data should be in pandas.DataFrame format
    columns=["fips", "series_complete_pop_pct"], # columns from data
    key_on="feature.properties.GEOID", # key from geo_data that matches the first column given in the columns parameter
    fill_color="Greens", # colorbrewer
    fill_opacity=0.6,
    line_opacity=0.3,
    
    # use the following lines if coloring based on quantile/bins
    bins=bins,
    legend_name="Vaccination Rate (%)"
).add_to(m)

# tooltips
county_layer.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['recip_state','recip_county','series_complete_pop_pct'], 
                                   aliases=['State','County','Fully Vaccinated Ppl%'],
                                   style=('background-color: grey; color: white;')
                                   )
).add_to(m)

## save the map
# m.save("CountyVaccinationMap.html")